# Purpose of this project

* ## Is ther any kind of correlaion between the drama script and content completion rate? 

---
---
---

https://www.whats-on-netflix.com/news/are-completion-rates-key-to-netflix-cancelations/
<img src="./image/1.PNG" width="700" />
<img src="./image/3.PNG" width="700" />

---
---
---

# Can we measure the completion rate through the transcript?

## 1. Gather two transcript data 
* The Irregulars : the lowest completion rate in upper image
* Squid Game : the highest completion rate in upper image > This is what I'm going to present 
 

## 2. Get dialogues without actions
* dropout actions : measure the depth between the characters using network analysis, not mise en scene(in the scene)
* make dataframe : make a dataframe with charector and dialogue

 

## 3. Measure the depth of emotions 
* run a network analysis : Using the dataframe, do a network analysis with each of episode of two drama-The Irregulars and Squid Game
* Compare the firues 

---
---
---

## Step 1. Gather two transcript data 
* The Irregulars : the lowest completion rate in upper image
* Squid Game : the highest completion rate in upper image > This is what I'm going to present 

In [1]:
# install used pakages 
# pip install selenium

In [2]:
# import modules I'm going to use

import selenium
import numpy as np
import pandas as pd

# for automation in further data gathering, I used selenium to crawle the transcript
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Here is a useful link about the usage of selenium!
# https://jackerlab.com/python-selenium/

In [3]:
def dynamic_web_crawler(url, xpath):
    """
    function name : dynamic_web_crawler
    input parameter : url, xpath 
    output parameter : crawled data
    """

    # to avoid unexpected exception, I used try-catch-finally 
    try:
        # go to web page
        driver.get(url)

        # wait until the transcript page id loaded 
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

        # find the element by xpath in html
        # which contains the transcript data
        elements = driver.find_elements(By.XPATH, xpath)
        # extract the text data
        data = [element.text for element in elements]
        # if all done, return the text data 
        return data
    except:
        # if error comes up while running the code, shut down the driver
        print("error occures")
        driver.quit()
    finally:
        # if all done, shut down the driver 
        driver.quit()
        

In [4]:
# Before calling the function, defind the input parameters 
url = 'https://scrapsfromtheloft.com/tv-series/squid-game-s01e09-one-lucky-day-transcript/'
xpath = '//*[@id="main"]/div/section/div/div/div/div[7]/div'

In [5]:
# define the webdriver as driver to open Chrome browser
driver = webdriver.Chrome()

In [6]:
# call the function I made
script = dynamic_web_crawler(url, xpath)

error occures


In [7]:
# check the return value of the function 
print("type of script variable is", type(script), '\n')
print("length of script variable is", len(script), '\n')
print("contents of script variable is \n", script)

type of script variable is <class 'NoneType'> 



TypeError: object of type 'NoneType' has no len()

## Step 2. Get dialogues without actions
* dropout actions : measure the depth between the characters using network analysis, not mise en scene(in the scene)
* make dataframe : make a dataframe with charector and dialogue

## make a function named 'pp1'(preprocessing1)
* delete the text that matches the condition
    * first line of the script : release date
    * second paragraph of the transcript : summary of the episode 
    * the actual transcript starts with the three star marks (* * *)


<img src="./image/4.PNG" width="700" />

In [ ]:
def pp1(script) :  
    """
    function name : pp1
    input parameter : script
    output parameter : preprocessed script(erase the text which is not script)
    """

    # first line of the transcript : release date,  
    full_script = script[0]  
    # second line(paragraph) of the transcript : summary of the episode
    full_script = full_script.split('\n', 2)[2]
    # the actual transcript starts with the three star marks (* * *)
    full_script = full_script.split('*', 3)[3]
    return full_script

In [ ]:
# call pp1
pp1_script = pp1(script)

In [ ]:
# result of pp1 
pp1_script 

## make a function named 'pp2'(preprocessing2)
* dropout the actions in the script
    * split the text data with the newline delimeter('\n[')
    * if there is just a bracelet(ex. []) with no other line, then that's an Action(in korean 지문)
    * if there is a bracelet with some lines, then that is a dialogue(in korean 대사)
        * in this case, the words in bracelet is the character's name 

<img src="./image/5.jpg" width="700" />

In [ ]:
def isDialogue(line) : 
    """
    function name : isDialogue
    input parameter : line
    output parameter : Boolean
    """
    bracelet_idx = line.find(']')
    line_len = len(line)
    # check if line has no words
    if (line_len < 1) |\
        ((bracelet_idx>0) & ((bracelet_idx+1)==line_len)):
        #check if there is just action with braclet 
        # then, it is an action so return False 
        return False 
    else :  
        return True

In [ ]:
def pp2(script) :  
    """
    function name : pp2
    input parameter : script
    output parameter : preprocessed script(dropout Actions)
    """
    # split the text data with the newline delimeter('\n[')
    # I used the delimeter '\n[' because there is newline('\n') between some dialogues 
    full_script = script.split('\n[')
    # using list comprehension, dropout actions, gather dialogues
    full_script = [line for line in full_script if isDialogue(line)]
    return full_script

In [ ]:
# pp2(pp1_script)
pp2_script = pp2(pp1_script)

In [ ]:
pp2_script

* I'll use the function name 'pp3'(preprocessing3)
    * split the text data with the newline delimeter(']')
    * put this data in DataFrame (columns = ['character', 'dialogue'])

In [ ]:
def pp3(script) :
    """
    function name : pp3
    input parameter : script
    output parameter : dataframe of script
    """    
    # using split methods, devide the line into character and diagloue
    # and make it as an pandas dataframe 
    script_df = pd.DataFrame({  'character':[line.split(']')[0].strip() for line in script]
                              , 'dialogue':[line.split(']')[1].lstrip().replace('\n', '') for line in script]})
    return script_df

In [ ]:
pp3_script = pp3(pp2_script)

In [ ]:
pp3_script.head(10)

## make a function named 'modify_dialogue'(modify_dialogue)
* There are some errors in the dataFrame
    * some line don't follow the rules 
    * find the irregular one and modify like below 

<img src="./image/6.jpg" width="500" />

In [ ]:
#check the dataframe column 'character' which does not have a character's name 
pp3_script['character'].value_counts().index.to_list()

In [ ]:
# extract some actions in character column
notcharectors_list = ['Korean', 'English', 'gasps', 'grunts', 'grunts', 'inhales'
               , 'angrily', 'automated']


# join the noncharacters_list with join function 
nochar_str = '|'.join(notcharectors_list)

In [ ]:
# using the condition I made in upper cell(nonchar_str), get the rows to be modifed 
redo_index = pp3_script[pp3_script['character'].str.contains(nochar_str)].index

In [ ]:
def modify_dialogue(line):
    """
    function name : modify_dialogue
    input parameter : line
    output parameter : None 
    """    
    print("!")
    # get the index of the parameter
    append_tar = line.index
    # using + operator, concat the problem line with the line before the problem line
    for i in append_tar:
        pp3_script.loc[i-1, 'dialogue'] = pp3_script.loc[i-1, 'dialogue'] + pp3_script.loc[i, 'dialogue']
        


In [ ]:
# call modify_dialogue function
modify_dialogue(pp3_script[pp3_script['character'].str.contains(nochar_str)])

In [ ]:
# pd.options.display.max_colwidth = 2000
# check the result 
pp3_script['dialogue'].head(11)

In [ ]:
# remove the action line in the dataframe
pp3_script.drop(redo_index).reset_index(inplace=True)

In [ ]:
# check the result 
pp3_script.head(11)